In [15]:
import cv2
import os
import keras
from keras.applications.vgg16 import VGG16
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras import backend as K
from keras import optimizers
import matplotlib.pyplot as plt
import numpy as np
K.clear_session()

In [3]:
def preprocess_images():
    path_v_r_r = './data/validation_original/realfoggy/'
    path_v_s_r = './data/validation_original/syntheticfoggy/'
    path_t_r_r = './data/train_original/realfoggy/'
    path_t_s_r = './data/train_original/syntheticfoggy/'
    paths_read = [path_v_r_r, path_v_s_r, path_t_r_r, path_t_s_r]
    path_v_r_w = './data/validation/realfoggy/'
    path_v_s_w = './data/validation/syntheticfoggy/'
    path_t_r_w = './data/train/realfoggy/'
    path_t_s_w = './data/train/syntheticfoggy/'
    paths_write = [path_v_r_w, path_v_s_w, path_t_r_w, path_t_s_w]
    labels = ['realfoggy', 'syntheticfoggy', 'realfoggy', 'syntheticfoggy']

    for (path_read, path_write, label) in zip(paths_read, paths_write, labels):
        counter = 0
        for filename in os.listdir(path_read):
            img = cv2.imread(os.path.join(path_read,filename))
            img = cv2.resize(img,(398, 224))
            img = img[:, 87:311, :]
            cv2.imwrite(path_write + label + str(counter) + '.jpg', img)
            counter += 1
    # if we want to show any of the images
    # plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

In [4]:
# https://colab.research.google.com/drive/1s1lkOiqKgl9_MsXJtPF--oh6uXHYkNYm#scrollTo=6IgZ7G0r4mUt
def change_model(model, new_input_shape=(None, 40, 40, 3)):
    # replace input shape of first layer
    model._layers[0].batch_input_shape = new_input_shape

    # rebuild model architecture by exporting and importing via json
    new_model = keras.models.model_from_json(model.to_json())

    # copy weights from old model to new one
    for layer in new_model.layers:
        try:
            layer.set_weights(model.get_layer(name=layer.name).get_weights())
            print("Loaded layer {}".format(layer.name))
        except:
            print("Could not transfer weights for layer {}".format(layer.name))

    return new_model

In [5]:
# load model without classifier layers
extractor = VGG16(include_top=False, input_shape=(224, 224, 3), weights='imagenet')
# add new classifier layers
model = Sequential()
model.add(extractor)
model.add(Flatten(input_shape=model.output_shape[1:]))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1024, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
for layer in extractor.layers[:25]:
    layer.trainable = False
# define new model
# summarize
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 1024)              25691136  
_________________________________________________________________
dropout_1 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_2 (Dense)              (None, 1024)              1049600   
_________________________________________________________________
dropout_2 (Dropout)  

In [6]:
# compile the model with a SGD/momentum optimizer
# and a very slow learning rate.
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])

In [7]:
# dimensions of our images.
img_width, img_height = 224, 224

train_data_dir = 'data/train'
validation_data_dir = 'data/validation'
nb_train_samples = 2 * 3088
nb_validation_samples = 2 * 720
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [8]:
batch_size = 16
# prepare data augmentation configuration
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        train_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_data_dir,
        target_size=(img_height, img_width),
        batch_size=batch_size,
        class_mode='binary')

# fine-tune the model
model.fit_generator(
        train_generator,
        steps_per_epoch=nb_train_samples // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=nb_validation_samples // batch_size)

Found 6176 images belonging to 2 classes.
Found 1440 images belonging to 2 classes.
Instructions for updating:
Use tf.cast instead.
Epoch 1/50
386/386 [==============================] - 1483s 4s/step - loss: 0.3811 - acc: 0.8352 - val_loss: 0.1783 - val_acc: 0.9472
Epoch 2/50
386/386 [==============================] - 1477s 4s/step - loss: 0.1440 - acc: 0.9649 - val_loss: 0.0943 - val_acc: 0.9771
Epoch 3/50
386/386 [==============================] - 1462s 4s/step - loss: 0.0911 - acc: 0.9775 - val_loss: 0.0676 - val_acc: 0.9840
Epoch 4/50
386/386 [==============================] - 1462s 4s/step - loss: 0.0623 - acc: 0.9848 - val_loss: 0.0512 - val_acc: 0.9861
Epoch 5/50
386/386 [==============================] - 1463s 4s/step - loss: 0.0510 - acc: 0.9879 - val_loss: 0.0433 - val_acc: 0.9889
Epoch 6/50
386/386 [==============================] - 1462s 4s/step - loss: 0.0428 - acc: 0.9880 - val_loss: 0.0349 - val_acc: 0.9889
Epoch 7/50
386/386 [==============================] - 1462s 4s/s

KeyboardInterrupt: 

In [9]:
model.save_weights('weights.h5')